In [1]:
# importing libraries
import json
import os
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [17]:
# loading games_reviews.json file using the json library
with open('games_reviews.json', 'r') as handle:
    data = [json.loads(line) for line in handle]
 
# creating lists for sentences,labels and urls
sentences = [] # headlines
labels = [] # labels
 
# iterating through the json data and loding 
# the requisite values into our python lists
for item in data:
    sentences.append(item['reviewText'])
    if item["overall"] > 3:
      labels.append(1)
    else:
      labels.append(0)

In [18]:

training_size = 134078

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]

training_labels = labels [0:training_size]
testing_labels = labels [training_size:]

In [19]:

vocab_size = 10000
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
# fitting tokenizer only to training set
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

# creating training sequences and padding them
traning_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(traning_sequences,maxlen = max_length,
                                padding = padding_type,
                                truncating=trunc_type,
                                )

# creating  testing sequences and padding them using same tokenizer
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen = max_length,
                                padding = padding_type,
                                truncating=trunc_type,
                                )

In [20]:
import numpy as np
# converting all variables to numpy arrays, to be able to work with tf version 2
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [21]:

embedding_dim = 16

# creating a model for sentiment analysis
model  = tf.keras.Sequential([
                # addinging an Embedding layer for Neural Network to learn the vectors
                tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
                # Global Average pooling is similar to adding up vectors in this case
                tf.keras.layers.GlobalAveragePooling1D(),
                tf.keras.layers.Dense(24, activation = 'relu'),
                tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [22]:
num_epochs = 10

history = model.fit(training_padded,training_labels, epochs = num_epochs,
                    validation_data = (testing_padded,testing_labels))

Epoch 1/10
4190/4190 [==============================] - 26s 6ms/step - loss: 0.4375 - accuracy: 0.8015 - val_loss: 0.3880 - val_accuracy: 0.8361
Epoch 2/10
4190/4190 [==============================] - 27s 6ms/step - loss: 0.3724 - accuracy: 0.8346 - val_loss: 0.3669 - val_accuracy: 0.8394
Epoch 3/10
4190/4190 [==============================] - 24s 6ms/step - loss: 0.3588 - accuracy: 0.8412 - val_loss: 0.3695 - val_accuracy: 0.8394
Epoch 4/10
4190/4190 [==============================] - 25s 6ms/step - loss: 0.3502 - accuracy: 0.8451 - val_loss: 0.3753 - val_accuracy: 0.8393
Epoch 5/10
4190/4190 [==============================] - 27s 6ms/step - loss: 0.3421 - accuracy: 0.8486 - val_loss: 0.3746 - val_accuracy: 0.8391
Epoch 6/10
4190/4190 [==============================] - 24s 6ms/step - loss: 0.3351 - accuracy: 0.8523 - val_loss: 0.3806 - val_accuracy: 0.8376
Epoch 7/10
4190/4190 [==============================] - 24s 6ms/step - loss: 0.3282 - accuracy: 0.8555 - val_loss: 0.3892 - val_ac

In [23]:
# forming new sentences for testing, feel free to experiment
# sentence 1 is bit sarcastic, whereas sentence two is a general statment.
new_sentence = [
                "This game is just filled with bugs. Completely garbage",
                "This might be GOTY. Most fun had with my family."]

# Converting the sentences to sequences using tokenizer
new_sequences = tokenizer.texts_to_sequences(new_sentence)
# padding the new sequences to make them have same dimensions
new_padded = pad_sequences(new_sequences, maxlen = max_length,
                           padding = padding_type,
                           truncating = trunc_type)

new_padded = np.array(new_padded )

print(model.predict(new_padded))

1/1 [==============================] - 0s 115ms/step
[[0.11798449]
 [0.9214925 ]]
